In [3]:
%pip install torch
import torch


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: C:\Users\fkonr\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


In [4]:
with open("input.txt", "r", encoding="utf-8") as file:
    text = file.read()
    
print(text[0:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [5]:
# These are the unique characters in the text

chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [6]:
# For small GPT we will use character size tokenizer, but in production, mostly using sub-work tokenizer, e.g.: tiktonizer

# enumerate function creates a tuple from the characters and their indices
# index:character - create a dictionary from it
stoi = { character:index for index,character in enumerate(chars) }
itos = { index:character for index,character in enumerate(chars) }

print(stoi)

{'\n': 0, ' ': 1, '!': 2, '$': 3, '&': 4, "'": 5, ',': 6, '-': 7, '.': 8, '3': 9, ':': 10, ';': 11, '?': 12, 'A': 13, 'B': 14, 'C': 15, 'D': 16, 'E': 17, 'F': 18, 'G': 19, 'H': 20, 'I': 21, 'J': 22, 'K': 23, 'L': 24, 'M': 25, 'N': 26, 'O': 27, 'P': 28, 'Q': 29, 'R': 30, 'S': 31, 'T': 32, 'U': 33, 'V': 34, 'W': 35, 'X': 36, 'Y': 37, 'Z': 38, 'a': 39, 'b': 40, 'c': 41, 'd': 42, 'e': 43, 'f': 44, 'g': 45, 'h': 46, 'i': 47, 'j': 48, 'k': 49, 'l': 50, 'm': 51, 'n': 52, 'o': 53, 'p': 54, 'q': 55, 'r': 56, 's': 57, 't': 58, 'u': 59, 'v': 60, 'w': 61, 'x': 62, 'y': 63, 'z': 64}


In [7]:
# encode and decode funtions for the tokenization

encode = lambda s: [stoi[c] for c in s] # input is a string, output is a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # input is a list of integer, output is a string

print(encode("abc"))
print(decode([39, 40, 41]))

[39, 40, 41]
abc


In [8]:
# Encoding the entire text dataset and store it into a torch.Tensor

print(torch.__version__)

data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

2.3.0+cpu
torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [9]:
# Let's split up the data into train and validation sets

n = int(0.9 * len(data)) # 90% of the data is for training
train_data = data[:n]
val_data = data[n:]

print(len(train_data), len(val_data))

1003854 111540


In [10]:
# We will train the dataset on chunks of datasets

block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [11]:
# This chunk of code illustrates how the probability should work: 
# If the first character is 18, then the 47 should follow based on the 'x'

x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [32]:
# Set seed for random number generator
torch.manual_seed(1337)

batch_size = 4 # How many independent sequences will we process in parallel
block_size = 8 # What is the maximum context length for predictions

# generate a small batch of data of inputs x and targets y
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,)) # gives back a tensor with random indices in the dataset with the size of back_size
    x = torch.stack([data[i : i+block_size] for i in ix]) # stack - https://pytorch.org/docs/stable/generated/torch.stack.html
    y = torch.stack([data[i+1 : i+block_size+1] for i in ix])
    return x,y

xb, yb = get_batch('train')
print('Inputs:')
print(xb.shape)
print(xb)

print('Targets:')
print(yb.shape)
print(yb)

Inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
Targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])


In [46]:
# Let's start implement the simplest neural network:

import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    
    def __init__(self, vocab_size):
        super().__init__()
        # Each token directly reads off the logits from the next token from a lookup table.
        # Embedding is a wrapper table - https://medium.com/@gautam.e/what-is-nn-embedding-really-de038baadd24
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size) # creates a vocab_size X vocab_size table
        
    def forward(self, idx, targets):
        # idx and targets are both (B, T) tensor of integers
        # logits are basically the scores
        logits = self.token_embedding_table(idx) # (Batch, Time, Channels)
        
        B, T, C = logits.shape
        logits = logits.view(B*T, C)
        targets = targets.view(B*T)
        
        # loss function - measure how far the logits are from the targets
        loss = F.cross_entropy(logits, targets) # cross_entropy would like to have the shape of the logits is (B, C, T)
        
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _in range(max_new_tokens)

In [47]:
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)
